In [1]:
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [2]:
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor

training_data = datasets.FashionMNIST(root="data", train=True,download=True,transform=ToTensor())

test_data = datasets.FashionMNIST(root="data", train=False,download=True,transform=ToTensor())

In [23]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(training_data, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=64, shuffle=True)

In [25]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [26]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        X = X.to(device)    
        y = y.to(device)
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            X = X.to(device)
            y = y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= size
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [27]:
model = NeuralNetwork().to(device)

loss_fn = nn.CrossEntropyLoss().to(device)
learning_rate = 1e-3
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [39]:
import time
start = time.process_time()
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")
elapsed = time.process_time() - start
print(elapsed)

Epoch 1
-------------------------------
loss: 2.267664  [    0/60000]
loss: 2.255674  [ 6400/60000]
loss: 2.247961  [12800/60000]
loss: 2.245626  [19200/60000]
loss: 2.211425  [25600/60000]
loss: 2.278155  [32000/60000]
loss: 2.269533  [38400/60000]
loss: 2.245637  [44800/60000]
loss: 2.230669  [51200/60000]
loss: 2.263699  [57600/60000]
Test Error: 
 Accuracy: 31.3%, Avg loss: 0.035343 

Epoch 2
-------------------------------
loss: 2.261657  [    0/60000]
loss: 2.247902  [ 6400/60000]
loss: 2.261286  [12800/60000]
loss: 2.254019  [19200/60000]
loss: 2.238542  [25600/60000]
loss: 2.243071  [32000/60000]
loss: 2.225893  [38400/60000]
loss: 2.230182  [44800/60000]
loss: 2.255391  [51200/60000]
loss: 2.256908  [57600/60000]
Test Error: 
 Accuracy: 31.4%, Avg loss: 0.035271 

Epoch 3
-------------------------------
loss: 2.225970  [    0/60000]
loss: 2.239163  [ 6400/60000]
loss: 2.260407  [12800/60000]
loss: 2.226233  [19200/60000]
loss: 2.265757  [25600/60000]
loss: 2.242791  [32000/600